In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool, cv
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns